In [133]:
print("hi")

import os
import pandas as pd
import subprocess
import re
import numpy as np
import json
import csv

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from pptx import Presentation
from pptx.util import Inches
import os

from scripts import combat_info
from scripts import combat_quick_apply
from scripts import combat_quick_QC


CAMCAN = "./DONNES/CamCAN.md.raw.csv.gz"
COMPILATION = "./DONNES/adni_compilation.csv.gz"

MAINFOLDER = "ROBUST"

RAWFOLDER = "RAW"

site_group = 'ADNI'
robust_method = 'IQR'
metric = "md"
method= "classic"

hi


In [134]:
def split_train_test(file_path, test_size=0.2, random_state=None):
    """
    Split the DataFrame into training and testing sets, ensuring the same proportion of HC and non-HC patients
    and that data from the same sid are in the same dataset.

    Parameters:
    file_path (str): The path to the CSV file to split.
    test_size (float): The proportion of the dataset to include in the test split.
    random_state (int): Random seed for reproducibility.

    Returns:
    pd.DataFrame: Training set.
    pd.DataFrame: Testing set.
    """
    df = pd.read_csv(file_path)
    
    # Group by 'sid' and get unique sids
    unique_sids = df.groupby('sid').first().reset_index()
    
    # Split the unique sids into train and test sets
    train_sids, test_sids = train_test_split(unique_sids, test_size=test_size, random_state=random_state, stratify=unique_sids['disease'])
    
    # Create train and test DataFrames by filtering the original DataFrame
    train_df = df[df['sid'].isin(train_sids['sid'])]
    test_df = df[df['sid'].isin(test_sids['sid'])]
    
    return train_df, test_df

In [135]:
def sample_patients(df, num_patients, disease_ratio,index):
    # Lire le fichier CSV dans un DataFrame
    
    # Calculer le nombre de patients malades et sains
    num_diseased = int(num_patients * disease_ratio)
    num_healthy = num_patients - num_diseased
    
    # Filtrer les patients en santé (HC) et malades
    healthy_patients = df[df['disease'] == 'HC']
    diseased_patients = df[df['disease'] != 'HC']
    
    # S'assurer qu'il y a assez de patients pour chaque catégorie
    if len(healthy_patients['sid'].unique()) < num_healthy or len(diseased_patients['sid'].unique()) < num_diseased:
        raise ValueError("Nombre insuffisant de patients en santé ou malades pour l'échantillon demandé.")
    
    # Sélectionner un échantillon aléatoire de patients sains et malades
    sampled_healthy = healthy_patients.groupby('sid').sample(frac=1).head(num_healthy * df['bundle'].nunique())
    sampled_diseased = diseased_patients.groupby('sid').sample(frac=1).head(num_diseased * df['bundle'].nunique())
    
    # Combiner les échantillons pour obtenir le DataFrame final
    sampled_df = pd.concat([sampled_healthy, sampled_diseased])
    # Modifier les valeurs de 'site' pour toutes les lignes
    sampled_df['site'] = f"{num_patients}_patients_{int(disease_ratio*100)}_percent_{index}"
    
    # Retourner le DataFrame final
    return sampled_df

In [136]:
def generate_biaised_data(df1, df2, 
                additive_uniform_low=-4, additive_uniform_high=4, 
                multiplicative_uniform_low=0.5, multiplicative_uniform_high=2, 
                additive_std_low=0.01, additive_std_high=0.1, 
                multiplicative_std_low=0.01, multiplicative_std_high=0.1):
    """
    Génère des biais additifs et multiplicatifs pour chaque bundle en fonction de df1, puis applique ces biais à df1 et df2
    de manière indépendante en tenant compte des covariables (âge, sexe, latéralité) et en centrant les résidus.

    Parameters:
    - df1, df2 (pd.DataFrame): Les DataFrames sur lesquels appliquer les biais.
    - additive_uniform_low, additive_uniform_high : paramètres pour le biais additif.
    - multiplicative_uniform_low, multiplicative_uniform_high : paramètres pour le biais multiplicatif.
    - additive_std_low, additive_std_high : paramètres pour l'écart-type du biais additif.
    - multiplicative_std_low, multiplicative_std_high : paramètres pour l'écart-type du biais multiplicatif.

    Returns:
    - tuple : Deux DataFrames avec les biais appliqués indépendamment.
    """
    
    # Dictionnaires pour stocker les biais par bundle
    additive_bias_per_bundle = {}
    multiplicative_bias_per_bundle = {}

    # # Tirer les moyennes de biais de distributions uniformes pour le bundle
    additive_mean = np.random.uniform(low=additive_uniform_low, high=additive_uniform_high)
    multiplicative_mean = np.random.uniform(low=multiplicative_uniform_low, high=multiplicative_uniform_high)
    
    # # Tirer les écarts-types de biais de distributions uniformes pour le bundle
    additive_std = np.random.uniform(low=additive_std_low, high=additive_std_high)
    multiplicative_std = np.random.uniform(low=multiplicative_std_low, high=multiplicative_std_high)

    # Calcul des biais pour chaque bundle unique dans df1
    for bundle in df1['bundle'].unique(): 
        # Générer un biais additif et multiplicatif spécifique au bundle
        additive_bias_per_bundle[bundle] = np.random.normal(loc=additive_mean, scale=additive_std)
        multiplicative_bias_per_bundle[bundle] = np.random.normal(loc=multiplicative_mean, scale=multiplicative_std)
   
    # Appliquer les biais indépendamment à df1 et df2 en utilisant les mêmes biais générés
    biased_df1 = apply_bias(df1, additive_bias_per_bundle, multiplicative_bias_per_bundle)
    biased_df2 = apply_bias(df2, additive_bias_per_bundle, multiplicative_bias_per_bundle)
    
    return biased_df1, biased_df2, additive_bias_per_bundle, multiplicative_bias_per_bundle, [additive_mean, multiplicative_mean, additive_std, multiplicative_std]

def apply_bias(dataframe, additive_bias_per_bundle, multiplicative_bias_per_bundle):
    biased_df = dataframe.copy()
    biased_means = []
    
    # Application de la régression et des biais pour chaque bundle unique
    for bundle in biased_df['bundle'].unique():
        # Filtrer le DataFrame pour le bundle actuel
        bundle_df = biased_df[biased_df['bundle'] == bundle]
        
        # Préparer les covariables pour la régression
        X = bundle_df[['age', 'sex', 'handedness']]
        y = bundle_df['mean']
        
        # Ajuster le modèle de régression linéaire pour le bundle
        model = LinearRegression()
        model.fit(X, y)
        
        # Calculer les prédictions et les résidus pour le bundle
        predicted_mean = model.predict(X)
        residuals = y - predicted_mean

        # Récupérer les biais pour le bundle actuel
        additive_bias = additive_bias_per_bundle[bundle]
        multiplicative_bias = multiplicative_bias_per_bundle[bundle]
        
        # Appliquer les biais aux résidus centrés et réintégrer les effets des covariables
        biased_means_bundle = residuals * multiplicative_bias + additive_bias * np.std(y) + predicted_mean
        biased_means.extend(biased_means_bundle)
    
    # Assigner les valeurs biaisées calculées au DataFrame
    biased_df['mean'] = biased_means
    return biased_df


In [137]:
def get_info(mov_data_file):
    [df,bundles] = combat_info.info(mov_data_file)
    nb_hc = int(re.findall('HC\(n=(\d+)',df["DetailInfos"]["Disease"])[0])
    nb_total = df["DetailInfos"]["Number of Subject"]
    nb_sick = nb_total - nb_hc
    return [nb_total,nb_hc,nb_sick]

In [138]:
def get_bundles(mov_data_file):
    return combat_info.get_bundles(mov_data_file)

In [139]:
def robust_text(x):
    return "NoRobust" if x == 'No' else x

def rwp_text(x):
    return "RWP" if x else "NoRWP"
def get_site(mov_data_file):
    mov_data = pd.read_csv(mov_data_file)
    return mov_data.site.unique()[0]


In [140]:
def fit(mov_data_file, robust, rwp, directory, hc,):
    ###########
    ### fit ###
    ###########
    output_model_filename = (
            get_site(mov_data_file)
            + "."
            + metric
            + "."
            + method
            + "."
            + robust_text(robust)
            + "."
            + rwp_text(rwp)
            + ".model.csv"
        )
    cmd = (
        "scripts/combat_quick_fit.py"
        + " "
        + CAMCAN
        + " "
        + mov_data_file
        + " --out_dir "
        + directory
        + " --output_model_filename "
        + output_model_filename
        + " --method "
        + method
        + " --robust "
        + robust
        + " -f "
    )
    if rwp:
        cmd += ' --rwp'
    if hc: 
        cmd += ' --hc'
    subprocess.call(cmd, shell=True)
    return output_model_filename

In [141]:
def apply(mov_data_file, model_filename, robust, rwp, directory):
    output_filename = os.path.join(
            directory,
            get_site(mov_data_file)
            + "."
            + metric
            + "."
            + method
            + "."
            + robust_text(robust)
            + "."
            + rwp_text(rwp)
            + ".csv"
        )
    return output_filename, combat_quick_apply.apply(mov_data_file, model_filename, output_filename)

In [142]:
def visualize_harmonization(f, new_f, directory):
    cmd = (
        "scripts/combat_visualize_harmonization.py"
        + " "
        + CAMCAN
        + " "
        + f
        + " "
        + new_f
        + " --out_dir "
        + directory
        #+ " --bundles all"
        + " -f"
    )
    subprocess.call(cmd, shell=True)

In [143]:
def QC(output_filename, output_model_filename):
    return combat_quick_QC.QC(CAMCAN,output_filename, output_model_filename)

In [144]:
def create_presentation(directory):
    # Create a presentation object
    prs = Presentation()
    
    # Define the subdirectories
    subdirs = ["hc", "NoRobust", "robust", "robust_rwp"]
    # Get the list of images
    images = [img for img in os.listdir(os.path.join(directory, subdirs[0])) if method in img and img.endswith('.png')]
    
    for img in images:
        slide_layout = prs.slide_layouts[5]  # Use a blank slide layout
        slide = prs.slides.add_slide(slide_layout)
        
        for i, subdir in enumerate(subdirs):
            img_path = os.path.join(directory, subdir, img)
            left = Inches(0.5 + (i % 2) * 4.5)  # Positioning images in two columns
            top = Inches(0.2 + (i // 2) * 3.5)  # Positioning images in two rows with more space between rows
            
            # Add text above the image
            text_box = slide.shapes.add_textbox(left, top, width=Inches(4), height=Inches(0.5))
            text_frame = text_box.text_frame
            text_frame.text = subdir
            
            # Add the image
            slide.shapes.add_picture(img_path, left, top + Inches(0.5), width=Inches(4))
    
    # Save the presentation
    prs.save(os.path.join(directory, 'harmonization_results.pptx'))


In [145]:
def compare_distances(directory, site, hc_dists, no_robust_dists, robust_dists, robust_rwp_dists):
    comparison_results = {
        "hc_vs_no_robust": (np.array(hc_dists) - np.array(no_robust_dists))/np.array(no_robust_dists)*100,
        "robust_vs_no_robust": (np.array(robust_dists) - np.array(no_robust_dists))/np.array(no_robust_dists)*100,
        "robust_rwp_vs_no_robust": (np.array(robust_rwp_dists) - np.array(no_robust_dists))/np.array(no_robust_dists)*100
    }
    df = pd.DataFrame(comparison_results)
    
    # Calculer le nombre de comparaisons négatives et positives, et les moyennes et médianes
    results = []
    for method in comparison_results.keys():
        negative_values = df[method][df[method] < 0]
        positive_values = df[method][df[method] >= 0]
        
        num_negative = len(negative_values)
        num_positive = len(positive_values)
        
        mean_negative = negative_values.mean() if num_negative > 0 else 0
        mean_positive = positive_values.mean() if num_positive > 0 else 0
        
        median_negative = negative_values.median() if num_negative > 0 else 0
        median_positive = positive_values.median() if num_positive > 0 else 0
        
        mean_difference = df[method].mean()
        
        results.append({
            "site": site,
            "comparaison": method,
            "Nb comp. nég.": num_negative,
            "Nb comp. pos.": num_positive,
            "Moy. tot.": mean_difference,
            "Moy. val. nég.": mean_negative,
            "Moy. val. pos.": mean_positive,
            "Méd. val. nég.": median_negative,
            "Méd. val. pos.": median_positive
        })
    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(directory, f"{site}_comparison_results.csv"), index=False)
    return results_df


In [146]:
def harmonize(f_train, f_test, directory, robust, rwp,hc):
    os.makedirs(directory, exist_ok=True)
    print(f_train)
    
    # Fit the model
    output_model_filename = fit(f_train, robust, rwp, directory, hc)
    output_model_filename = os.path.join(directory, output_model_filename)
    # Apply the model
    output_filename, y_harm = apply(f_test, output_model_filename, robust, rwp, directory) 
    
    # Perform quality control
    dists = QC(output_filename, output_model_filename)
    
    # Visualize the harmonization
    visualize_harmonization(f_test, output_filename, directory)
    
    # If robust is not "No", load metrics and outliers
    if robust != "No":
        metrics_filename = os.path.join(directory, f"metrics_{get_site(f_train)}_{robust_text(robust)}_{rwp_text(rwp)}.csv")
        outliers_filename = os.path.join(directory, f"outliers_{get_site(f_train)}_{robust_text(robust)}_{rwp_text(rwp)}.csv")
        
        # Load metrics from CSV file
        loaded_metrics = pd.read_csv(metrics_filename, index_col=0)
        
        # Load outliers from CSV file
        loaded_outliers_df = pd.read_csv(outliers_filename, index_col=0)
        
        return [dists, loaded_metrics, loaded_outliers_df]
    return[dists, None, None]

In [147]:
def analyse_site(f_train,f_test, robust, directory):
    # 4 harmonization
    harmonization_hc = harmonize(f_train, f_test, os.path.join(directory, "hc"), "No", False, True)
    harmonization_no_robust = harmonize(f_train, f_test, os.path.join(directory, "NoRobust"), "No", False, False)
    harmonization_robust = harmonize(f_train, f_test, os.path.join(directory, "robust"), robust, False, False)
    harmonization_robust_rwp = harmonize(f_train, f_test, os.path.join(directory, "robust_rwp"), robust, True, False)


    create_presentation(directory)

    dists_analyze = compare_distances(directory, get_site(f_train), harmonization_hc[0], harmonization_no_robust[0], harmonization_robust[0], harmonization_robust_rwp[0])
    
    #TODO bundles et analyze outliers
    return dists_analyze, harmonization_robust[1], harmonization_robust[2]

In [148]:
# #Analyse Method
# #sample_sizes = [5,10,30, 50, 100, 150, 200,300]  # Différentes tailles d'échantillon
# #disease_ratios = [0.05, 0.1, 0.3, 0.5, 0.7]  # Différents pourcentages de malades
# sample_sizes = [30, 50, 150]  # Différentes tailles d'échantillon
# disease_ratios = [0.1, 0.3, 0.5]  # Différents pourcentages de malades
# num_tests = 2  # Nombre de tests à effectuer pour chaque combinaison
# # Split the data into training and testing sets
# directory = os.path.join(MAINFOLDER, robust_method)
# train_df, test_df = split_train_test(COMPILATION, test_size=0.2, random_state=42)
# # Initialize DataFrames to store the results
# metrics_compilation = pd.DataFrame()
# dists_compilation = pd.DataFrame()
# outliers_compilation = pd.DataFrame()
# for sample_size in sample_sizes:
#     for disease_ratio in disease_ratios:
        
#         sizeDir = os.path.join(directory, f"{sample_size}_{int(disease_ratio*100)}")
#         for i in range(num_tests):
#             tempDir = os.path.join(sizeDir, f"{i}")
#             os.makedirs(tempDir, exist_ok=True)
            
#             # Sauvegarder l'échantillon dans un fichier temporaire
#             temp_file = os.path.join(tempDir, f"train_{sample_size}_{int(disease_ratio*100)}_{i}.csv")

#             test_file = os.path.join(tempDir, f"test_{sample_size}_{int(disease_ratio*100)}_{i}.csv")
            
#             # Analyser le site pour le nouvel échantillon
#             dists_analyze, metrics, outliers = analyse_site(temp_file, test_file, robust_method, tempDir)
#             metrics_compilation = pd.concat([metrics_compilation, metrics])
#             dists_compilation = pd.concat([dists_compilation, dists_analyze])
#             outliers_compilation = pd.concat([outliers_compilation, outliers])
# # Save the metrics and distances compilation DataFrames to CSV files
# metrics_compilation.to_csv(os.path.join(directory, "metrics_compilation.csv"), index=False)
# dists_compilation.to_csv(os.path.join(directory, "dists_compilation.csv"), index=False)
# outliers_compilation.to_csv(os.path.join(directory, "outliers_compilation.csv"), index=False)

# dists_compilation['site'] = dists_compilation['site'].str.rsplit('_', n=1).str[0]
# metrics_compilation['site'] = metrics_compilation['site'].str.rsplit('_', n=1).str[0]

# # Display the means by site
# dists_means_by_site = dists_compilation.groupby(['site','comparaison']).mean()
# metrics_means_by_site = metrics_compilation.groupby('site').mean()

# metrics_means_by_site.to_csv(os.path.join(directory, "metrics_compilation_mean.csv"), index=False)
# dists_means_by_site.to_csv(os.path.join(directory, "dists_compilation_mean.csv"), index=False)
# print("FINI")

In [149]:
# REAL SITES
# directory = os.path.join(MAINFOLDER, robust_method)
# raw_directory = os.path.join(RAWFOLDER, site_group)
# for filename in sorted(os.listdir(raw_directory)):
#     f = os.path.join(raw_directory, filename)
#     # checking if it is a file
#     if os.path.isfile(f):
#         analyse_site(f, robust_method, directory)
        


In [150]:
# # TEST ADD BIAIS
# Split the data into training and testing sets
directory = os.path.join(MAINFOLDER, "testBiais")
os.makedirs(directory, exist_ok=True)
train_df, test_df = split_train_test(CAMCAN, test_size=0.2, random_state=42)

# Generate biased data
# Save the original non-biased data to temporary files
temp_train_file_original = os.path.join(directory, "temp_train_original.csv")
temp_test_file_original = os.path.join(directory, "temp_test_original.csv")
train_df.to_csv(temp_train_file_original, index=False)
test_df.to_csv(temp_test_file_original, index=False)

# Generate biased data
sampled_df_biaied, test_df_biaised, gammas,deltas, ruffles= generate_biaised_data(train_df, test_df)

# Save the biased data to temporary files
temp_train_file = os.path.join(directory, "temp_train_biased.csv")
temp_test_file = os.path.join(directory, "temp_test_biased.csv")
sampled_df_biaied.to_csv(temp_train_file, index=False)
test_df_biaised.to_csv(temp_test_file, index=False)

# Run the combat_visualize_data script
outname_train = os.path.join("visualize_train")
cmd = (
    "scripts/combat_visualize_data.py"
    + " "
    + temp_train_file_original
    + " "
    + temp_train_file
    + " --out_dir "
    + directory
    + " --outname "
    + outname_train
    + " -f"
    + " --bundles all"
)
subprocess.call(cmd, shell=True)

# Display gammas and deltas along with their mean and standard deviation
print("Gammas:", gammas)
print("Deltas:", deltas)
gammas = list(gammas.values())
deltas = list(deltas.values())
print("\nGamma Statistics:")
print(f"Mean: {np.mean(gammas)}, Std: {np.std(gammas)}")

print("\nDelta Statistics:")
print(f"Mean: {np.mean(deltas)}, Std: {np.std(deltas)}")
print("Ruffles:", ruffles)


Gammas: {'left_ventricle': -3.647586673874433, 'mni_AC': -3.6165479682646215, 'mni_AF_L': -3.679153972284244, 'mni_AF_R': -3.63622851447761, 'mni_AST_L': -3.6254036629122974, 'mni_AST_R': -3.555941602597745, 'mni_CC': -3.6465103682823976, 'mni_CCMid': -3.602065754135561, 'mni_CC_ForcepsMajor': -3.6650867394526063, 'mni_CC_ForcepsMinor': -3.7078917055163414, 'mni_CST_L': -3.5924880934879346, 'mni_CST_R': -3.6302430513318495, 'mni_C_L': -3.568313766823324, 'mni_C_R': -3.615818267950327, 'mni_FPT_L': -3.585697641968588, 'mni_FPT_R': -3.675533438298187, 'mni_F_L_R': -3.4952840217128833, 'mni_ICP_L': -3.7657620146791646, 'mni_ICP_R': -3.6382685858078934, 'mni_IFOF_L': -3.5668608652733322, 'mni_IFOF_R': -3.5645614130273118, 'mni_IIT_mask_skeletonFA': -3.549896306772501, 'mni_ILF_L': -3.6785982198609317, 'mni_ILF_R': -3.413291888446699, 'mni_MCP': -3.587989434808899, 'mni_ML_L': -3.571340541019628, 'mni_ML_R': -3.590313596084942, 'mni_MdLF_L': -3.563443611847447, 'mni_MdLF_R': -3.637486115985

In [151]:
# # TEST Powerpoint generation
# d  = os.path.join(MAINFOLDER, robust_method, "adni_100_Philips_3T")
# create_presentation(d)

In [152]:
# # TEST the sample_patients function with compilation data data
# sampled_df = sample_patients(COMPILATION, num_patients=100, disease_ratio=0.5)
# print(sampled_df)


In [153]:
# load_metrics("ROBUST/IQR/50_30/0/", "50_patients_30_percent_0")

In [154]:
# # Load the dists_compilation and metrics_compilation CSV files
# dists_compilation_path = os.path.join(directory, "dists_compilation.csv")
# metrics_compilation_path = os.path.join(directory, "metrics_compilation.csv")

# dists_compilation = pd.read_csv(dists_compilation_path)
# metrics_compilation = pd.read_csv(metrics_compilation_path)

# # Change the site column
# dists_compilation['site'] = dists_compilation['site'].str.rsplit('_', n=1).str[0]
# metrics_compilation['site'] = metrics_compilation['site'].str.rsplit('_', n=1).str[0]

# # Display the means by site
# dists_means_by_site = dists_compilation.groupby(['site','comparaison']).mean()
# metrics_means_by_site = metrics_compilation.groupby('site').mean()

# print(dists_means_by_site)
# print(metrics_means_by_site)